In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer, AdamW

In [ ]:
# Load pre-trained BERT model and tokenizer
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import re
def dataloader(dataset) :
    import pandas as pd
    import re  # Import the 're' module for regular expressions

    # Define the path to your CSV file
    csv_file_path = '/content/anomaly_label.csv'

    # Read the CSV file using pandas
    df = pd.read_csv(csv_file_path)

    # Define the function to generate labels based on "alias_list"
    def generate_labels(alias_list):
        search_pattern = r"Unknown host"
        if pd.notna(alias_list) and re.search(search_pattern, alias_list):
            return 0
        else:
            return 1

    # Apply the function to create the "labels" column
    df["labels"] = df["alias_list"].apply(generate_labels)

    # Save the modified DataFrame back to the CSV file
    df.to_csv(csv_file_path, index=False)

     # Specify the column you want to read
    target_column_name_1 = 'client'  # Replace with the name of the desired column
    target_column_name_2 = 'labels'



    labels=[]
    n= len(df)-2
    print(n)


    with open(dataset, 'r') as file:
        log_entries = file.readlines()

    for i in range(1,n):
        column_values_1 = df.loc[i,target_column_name_1]
        column_values_2 = df.loc[i,target_column_name_2]
        for log_entry in log_entries:
            # Define a regex pattern to match IP addresses
            ip_address_pattern = r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}'

            # Find the IP address using regex
            ip_address_match=re.search(ip_address_pattern, log_entry)
            ip_address = ip_address_match.group()

            if((column_values_1==ip_address) and (column_values_2==0)):
                labels.append(0)

            elif ((column_values_1==ip_address) and (column_values_2==1)):
                labels.append(1)

    #print(labels)

    max_sequence_length = 128


    texts = log_entries

    lol=[]
    lol.append(labels)

    # Tokenize and preprocess the dataset
    input_ids = []
    attention_masks = []

    for text in texts:
        encoded_text = tokenizer.encode_plus(
        text,
        max_length=max_sequence_length,
        padding='max_length',
        truncation=True,
        return_tensors='pt',
        )



    input_ids.append(encoded_text['input_ids'])
    print(input_ids)
    input_ids = torch.cat(input_ids, dim=0)

    attention_masks.append(encoded_text['attention_mask'])
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(lol)

    return  input_ids, attention_masks, labels

In [ ]:
x=dataloader(dataset="/content/train.log")
input_ids, attention_masks, labels=x
train_dataset = TensorDataset(input_ids, attention_masks, labels)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

258443
[tensor([[  101,  7558,  1012, 15376,  1012,  6356,  1012, 22884,  1011,  1011,
          1031,  2570,  1013,  5553,  1013, 10476,  1024,  5840,  1024,  6021,
          1024,  4090,  1009,  6021, 14142,  1033,  1000,  2131,  1013, 11307,
          1013, 26261,  9048, 12837,  1010,  1052, 18827,  2692,  2620,  2475,
          8299,  1013,  1015,  1012,  1015,  1000,  3263, 21036, 12521,  1000,
          1011,  1000,  1000,  9587,  5831,  4571,  1013,  1019,  1012,  1014,
          1006,  3645, 23961,  1020,  1012,  1015,  1025, 27634,  1024,  4413,
          1012,  1014,  1007, 16216, 19665,  1013,  2230, 24096, 24096,  2543,
         14876,  2595,  1013,  4413,  1012,  1014,  1000,  1000,  1011,  1000,
           102,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0, 

In [ ]:
# Define compliance labels
compliance_labels = {
    0: 'Non-Compliant',
    1: 'Compliant'
}

In [ ]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

# Define your optimizer
learning_rate = 0.001
#optimizer = optim.Adam(model.parameters(), lr=learning_rate)
# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=1e-5)
loss_fn = torch.nn.CrossEntropyLoss()
# Create an instance of your DataLoader using your preprocessed dataset
dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    for batch in dataloader:
        optimizer.zero_grad()
        inputs, attention_mask, labels = batch
        outputs = model(inputs, attention_mask)
        #loss = loss_fn(outputs, labels)  # You need to define your loss function
        #loss_fn.backward()
        optimizer.step()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Read the test log file
with open('test.log', 'r') as file:
    log_entries = file.readlines()
pred_label=[]
# Process each log entry and generate compliance predictions
for log_entry in log_entries:
    inputs = tokenizer(log_entry, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits).item()
    pred_label.append(predicted_label)
    compliance_label = list(compliance_labels.keys())[predicted_label]
    predicted_compliance = compliance_labels[predicted_label]

In [ ]:
#import torch

# Create a tensor
#tensor = torch.tensor([1, 2, 3, 4, 5])

# Convert tensor to a list
tensor_list = labels.tolist()

#print(tensor_list)  # Output: [1, 2, 3, 4, 5]

ground_truth_labels=[]
ground_truth_labels=tensor_list[0]

# Assuming you have lists of predicted_labels and true_labels
# Replace with actual predicted labels (0 or 1)
true_labels = ground_truth_labels  # Replace with actual true labels (0 or 1)

# Initialize counts for TP, TN, FP, and FN
tp = tn = fp = fn = 0

# Calculate TP, TN, FP, FN
for pred, true in zip(pred_label, true_labels):
    if pred == 1 and true == 1:
        tp += 1
    elif pred == 0 and true == 0:
        tn += 1
    elif pred == 1 and true == 0:
        fp += 1
    elif pred == 0 and true == 1:
        fn += 1

# Calculate accuracy, precision, and recall
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)

print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")

True Positives: 27
True Negatives: 5
False Positives: 86
False Negatives: 3
Accuracy: 0.26
Precision: 0.24
Recall: 0.90


In [ ]:
# Read the vad log file
with open('vad.log', 'r') as file:
    log_entries = file.readlines()
pred_label=[]
# Process each log entry and generate compliance predictions
for log_entry in log_entries:
    inputs = tokenizer(log_entry, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_label = torch.argmax(outputs.logits).item()
    pred_label.append(predicted_label)
    compliance_label = list(compliance_labels.keys())[predicted_label]
    predicted_compliance = compliance_labels[predicted_label]
    print(f"Log Entry: {log_entry}")
    print(f"Predicted Compliance: {compliance_label}\n")
    print(f"Predicted Compliance: {predicted_compliance}")
    print(pred_label)


Log Entry: 31.56.102.211 - - [22/Jan/2019:17:42:12 +0330] "GET /image/34290?name=galaxy-j2-core-1.jpg&wh=200x200 HTTP/1.1" 200 4328 "https://torob.com/search/?query=%D8%A7%D9%84%20%D8%B3%DB%8C%20%D8%AF%DB%8C%20%DA%AF%D9%84%DA%A9%D8%B3%DB%8C%20j2" "Mozilla/5.0 (Windows NT 6.3; Win64; x64; rv:64.0) Gecko/20100101 Firefox/64.0" "-"

Predicted Compliance: 0

Predicted Compliance: Non-Compliant
[0]
Log Entry: 5.123.144.95 - - [24/Jan/2019:23:53:52 +0330] "GET /image/33888?name=model-b2048u-1-.jpg&wh=200x200 HTTP/1.1" 200 3975 "-" "Dalvik/2.1.0 (Linux; U; Android 8.0.0; ALP-L29 Build/HUAWEIALP-L29S)" "-"

Predicted Compliance: 1

Predicted Compliance: Compliant
[0, 1]
